In [6]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attributes=[]):
        self.attributes = attributes
        return self
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attributes].values

def readData(filename):
    return pd.read_csv(filename)

def dropData(data, toDrop=[]):
    data = data.drop(columns=toDrop)
    return data

def cleanData(data, toFill=[]):
    return data

dataset = readData("train.csv")

dropList = ["Cabin", "Ticket", "Name", "PassengerId"]
fillList = ["Embarked"]
dataset = dropData(data=dataset, toDrop=dropList)
dataset = cleanData(data=dataset, toFill=fillList)
numCols = dataset.shape[1]

np.random.seed(42)

#Split the datset into train and test, then into features and labels
dataTrain, dataTest = train_test_split(dataset, test_size=.2)
labelsTrain = dataTrain.iloc[:, 0:0]
dataTrain = dataTrain.iloc[:, 1:]
labelsTest = dataTest.iloc[:, 0:0]
dataTest = dataTest.iloc[:, 1:]

In [4]:
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 331 to 102
Data columns (total 7 columns):
Pclass      712 non-null int64
Sex         712 non-null object
Age         572 non-null float64
SibSp       712 non-null int64
Parch       712 non-null int64
Fare        712 non-null float64
Embarked    710 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 44.5+ KB


In [16]:
median = dataTrain["Embarked"].mode

In [17]:
dataTrain = dataTrain.fillna(value={"Embarked": median})
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 331 to 102
Data columns (total 7 columns):
Pclass      712 non-null int64
Sex         712 non-null object
Age         572 non-null float64
SibSp       712 non-null int64
Parch       712 non-null int64
Fare        712 non-null float64
Embarked    712 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 44.5+ KB
